# Sequence Mining

Implementazione del **Sequence Mining**.

In [41]:
import nbimporter
from t1_ETLBasics import calories_to_df,sleep_to_df,exercise_to_df
from t3_TSToEvents import calories_to_events, sleep_to_events,exercise_to_events
from t4_FeatureRedaction import calories_kmeans, sleep_kmeans, exercise_kmeans
from t5_MakeTransactions import get_cal_transaction, get_sleep_transaction, get_exercise_transaction, get_master_transaction
import dateutil as du
from datetime import datetime
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import re
from operator import itemgetter
PATH = '../../pmdata/'

In [42]:
cal_transaction = get_cal_transaction(calories_kmeans(calories_to_events(calories_to_df(PATH,[1,2])),4))
sleep_transaction = get_sleep_transaction(sleep_kmeans(sleep_to_events(sleep_to_df(PATH,[1,2])),4))
ex_transaction = get_exercise_transaction(exercise_kmeans(exercise_to_events(exercise_to_df(PATH,[1,2])),4))

In [43]:
main_transaction = get_master_transaction(cal_transaction,ex_transaction,sleep_transaction)

In [35]:
main_transaction

[[[datetime.datetime(2019, 11, 1, 0, 0),
   'calories cluster: 1, weekend: False'],
  [datetime.datetime(2019, 11, 1, 1, 0),
   'calories cluster: 1, weekend: False'],
  [datetime.datetime(2019, 11, 1, 2, 0),
   'calories cluster: 1, weekend: False'],
  [datetime.datetime(2019, 11, 1, 3, 0),
   'calories cluster: 1, weekend: False'],
  [datetime.datetime(2019, 11, 1, 4, 0),
   'calories cluster: 1, weekend: False'],
  [datetime.datetime(2019, 11, 1, 5, 0),
   'calories cluster: 1, weekend: False'],
  [datetime.datetime(2019, 11, 1, 6, 0),
   'calories cluster: 3, weekend: False'],
  [datetime.datetime(2019, 11, 1, 7, 0),
   'calories cluster: 0, weekend: False'],
  [datetime.datetime(2019, 11, 1, 8, 0),
   'calories cluster: 1, weekend: False'],
  [datetime.datetime(2019, 11, 1, 9, 0),
   'calories cluster: 0, weekend: False'],
  [datetime.datetime(2019, 11, 1, 10, 0),
   'calories cluster: 0, weekend: False'],
  [datetime.datetime(2019, 11, 1, 11, 0),
   'calories cluster: 0, weekend:

In [36]:
def get_trans(trans):
    """Funzione che crea una lista contenente le transazioni raggruppate per data
    
    Parametri: 
    - trans: lista contenente le transazioni
    
    Ritorna:
    - lista che contiene le transazioni raggruppate per data e ordinate"""
    groupedByDay={}
    for t in trans:
        if t[0].date() in groupedByDay:
            groupedByDay[t[0].date()].append(t)
        else:
            groupedByDay[t[0].date()]=[t]
    return list(groupedByDay.values())

In [37]:
def get_list_m_transaction(main_transaction):
    """Funzione che ritorna una lista in cui un elemento
    è la lista delle transazioni raggruppate e ordinate 
    richiamando la funzione 'get_trans'.
    
    Parametri:
    - lista in cui l'i-esimo elemento è la lista di transazioni dell'i-esimo partecipante
    
    Ritorna:
    - lista in cui ogni elemento è la lista di transazioni ordinata e raggruppata per timestamp per l'i-esimo partecipante."""
    m_transaction = []
    for i in range(len(main_transaction)):
        m_transaction.append(get_trans(main_transaction[i]))
    return m_transaction

In [38]:
m_transaction = get_list_m_transaction(main_transaction)

In [ ]:
m_transaction

In [ ]:
def get_transaction(m_transaction):
    """Funzione che genera una lista di liste in cui un elemento è la lista completa delle transazioni per ogni partecipante.
        Viene eliminato il timestamp.
    
    Parametri:
    - m_transaction: lista delle transazioni in cui ho ancora il timestamp
    
    Ritorna:
    - transactions: lista di liste che contiene la lista delle transazioni per ogni partecipante senza timestamp."""
    transactions = []
    for m in m_transaction:
        transactions.append([ [i[1] for i in tr] for tr in m])
    return transactions

In [ ]:
transactions = get_transaction(m_transaction)

In [ ]:
transactions[0]

Implementazione utilizzando una struttura ad albero.

In [ ]:
def extract_unique_value(seq):  
    """Funzione che estrae i valori presenti nella seq
    
    Parametri:
    - seq: sequenza di transazioni
    
    Ritorna:
    - lista di elementi nella sequenza senza ripetizioni"""
    a = [] 
    for s in seq:
        a+=list(set(s))
    return list(set(a))

In [ ]:
def get_dictionary(trans):
    """ Funzione che genera il Dizionario per la nostra analisi
    
    Parametri: 
    - trans: lista delle transazioni
    
    Ritorna:
    - dictionary: lista degli elementi senza duplicati"""
    dictionary = extract_unique_value(trans)
    dictionary.sort(reverse=True)
    return dictionary

In [ ]:
def get_list_dictionaries(transactions):
    """ Funziona che genera una lista di liste in cui ogni elemento contiene il dizionario dell'i-esimo
    partecipante.
    
    Parametri:
    - transactions
    
    Ritorna:
    - dictionaries: lista di liste"""
    dictionaries = []
    for i in range(len(transactions)):
        d = get_dictionary(transactions[i])
        dictionaries.append(d)
    return dictionaries

In [ ]:
dictionaries = get_list_dictionaries(transactions)

In [ ]:
dictionaries[1]

Definire il **Separatore**:

In [ ]:
SEP = '<!>'

In [ ]:
def get_confidence(arr,trans,supp):
    """Funzione che calcola la confidenza di una sequenza:
    per esempio se ho [A,B,C,D] calcola la confudenza di {A,B,C} -> {D}
    
    Parametri:
    - arr: sequenza
    - trans: lista
    - supp: supporto di arr
    
    Ritorna:
    - la confidenza della sequenza"""
    antecedent = arr
    if(len(arr)>1):
        antecedent = arr[:-1]
    #sup_cons = get_supp(consequent,trans,1)
    sup_ant = get_supp(antecedent,trans)
    return supp/sup_ant

In [ ]:
def get_supp(d,t):
    """Funzione che calcola il supporto di una sequenza
    
    Parametri:
    - d: sequenza
    - t: transazioni
    
    Ritorna:
    - il supporto della sequenza sulla transazione"""
    c=0
    for tr in t:
        ttext = SEP + (SEP+SEP).join(tr) + SEP
        preg = '.*'.join([ re.escape(i) for i in d])
        search = re.search(preg, ttext)
        c = c + 1 if search else c
    return c/len(t)

In [ ]:

def extract_patterns_simple(t, minsup, minlen, maxlen,arr,dic):
    """ Funzione che stampa le sequenze che hanno un supporto maggiore del minsup e di lunghezza 
    tra minlen e maxlen.
    
    Parametri:
    - t: transazioni
    - minsup: supporto minimo
    - minlen: minima lunghezza della sequenza
    - maxlen: massima lunghezza della sequenza
    - arr: sequenza iniziale
    - dic: dizionario 
    
    Ritorna:
    - None"""
    for d in dic:
        arr.append(d)
        support = get_supp(arr,t)
        
        if  support >= minsup:
            if(len(arr) >= minlen and len(arr) <= maxlen):
                #test confidenza qua
                print(f'{arr}: Support: {support}')
                
            if(len(arr)<=maxlen):
                extract_patterns_simple(t, minsup, minlen, maxlen,arr,dic)
        arr.pop()


In [ ]:
#extract_patterns_simple(transactions,0.2,2,4,[],dictionary)

In [ ]:
for i in range(len(transactions)):
    print(f"\nPARTECIPANTE {i+1}\n*****************************\n\n")
    extract_patterns_simple(transactions[i],0.5,2,4,[],dictionaries[i])
    

In [ ]:
def check_confidence(conf,minconf):
    """ Controlla se la confidenza è minore della confidenza minima
    
    Parametri:
    - conf: confidenza trovata
    - minconf: minima confidenza
    
    Ritorna:
    - True se la conf è minore, False altrimenti."""
    if conf<minconf:
        return True
    return False

In [ ]:
def extract_patterns(t, minsup, minconf, minlen, maxlen,arr,dic,i):
    """ Funzione che estrae i pattern che hanno un supporto maggiore del minsup e una confidenza
    maggiore delle minconf e di lunghezza tra minlen e maxlen. Salvo in un file i pattern.
    
    Parametri:
    - t: transazioni
    - minsup: supporto minimo
    - minconf: minima confidenza
    - minlen: minima lunghezza della sequenza
    - maxlen: massima lunghezza della sequenza
    - arr: sequenza iniziale
    - dic: dizionario 
    
    Ritorna:
    - None"""
    for d in dic:
        arr.append(d)
        support = get_supp(arr,t)
        
        if  support >= minsup:
            if(len(arr) >= minlen and len(arr) <= maxlen):
                #test confidenza qua
                #print(f'{arr}: Support: {support}')
                
                confidence= get_confidence(arr,t,support)
                if(check_confidence(confidence,minconf)):
                    print('NESSUNA REGOLA')
                else:
                    print(f'{arr}: {confidence}')
                    # Qui si deve spostare la freccia a sinistra e ricalcolare la confidenza
                    #conf,ant = get_confidence(ant[:-1],t,get_supp(ant,t))
                    #check_confidence(conf,min_conf)
                    with open("./output"+"p{:02d}".format(i)+".txt", "a") as file:
                        for a in arr:
                            file.write(a+ "<!>")
                        file.write("\n")
            if(len(arr)<=maxlen):
                extract_patterns(t, minsup,minconf, minlen, maxlen,arr,dic,i)
        arr.pop()

In [25]:
#extract_patterns(transactions,0.2,0.9,2,4,[],dictionary)
for i in range(len(transactions)):
    print(f"\nPARTECIPANTE {i+1}\n*****************************\n\n")
    extract_patterns(transactions[i],0.5,0.96,2,4,[],dictionaries[i],i)

['calories cluster: 1, weekend: False', 'calories cluster: 3, weekend: False', 'calories cluster: 1, weekend: False']: 0.9722222222222222
NESSUNA REGOLA
NESSUNA REGOLA
NESSUNA REGOLA
['calories cluster: 1, weekend: False', 'calories cluster: 3, weekend: False', 'calories cluster: 0, weekend: False']: 0.962962962962963
NESSUNA REGOLA
NESSUNA REGOLA
['calories cluster: 1, weekend: False', 'calories cluster: 3, weekend: False', 'calories cluster: 0, weekend: False', 'calories cluster: 0, weekend: False']: 0.9807692307692307
['calories cluster: 1, weekend: False', 'calories cluster: 1, weekend: False']: 1.0
['calories cluster: 1, weekend: False', 'calories cluster: 1, weekend: False', 'calories cluster: 3, weekend: False']: 1.0
NESSUNA REGOLA
['calories cluster: 1, weekend: False', 'calories cluster: 1, weekend: False', 'calories cluster: 3, weekend: False', 'calories cluster: 1, weekend: False']: 0.9722222222222222
['calories cluster: 1, weekend: False', 'calories cluster: 1, weekend: Fal

In [26]:
def get_pattern(i):
    """Funzione che legge i file contenenti i pattern e li salva in una lista.
    
    Parametri:
    - i: contatore per leggere il file corretto
    
    Ritorna:
    -patterns1: lista dei pattern"""
    with open("./output"+ "p{:02d}".format(i)+".txt","r") as in_file:
        patterns = in_file.readlines()
    patt = []
    for p in patterns:
        patt.append(p.rstrip("\n"))
    patterns1 = []
    for p1 in patt:
        patterns1.append(p1.split("<!>"))
    for p2 in patterns1:
        p2.pop()
    return patterns1

In [27]:
def get_list_patterns(transactions):
    """Funzione che genera la lista di liste contenente la lista di patterns per ogni partecipante.
    
    Parametri:
    - transactions: transazioni
    
    Ritorna:
    - patterns: lista di liste"""
    patterns = []
    for i in range(len(transactions)):
        patterns.append(get_pattern(i))
    return patterns

In [28]:
patterns = get_list_patterns(transactions)

In [29]:
def get_association_rule(p,t,ant,i):
    """Funzione che genera le association rule e le stampa.
    
    Parametri:
    - p: lista dei pattern
    - t: transazioni
    - ant: antecedente
    - i: contatore
    
    Ritorna:
    - None"""
    conf = get_supp(p,t)/get_supp(ant,t)
    if(check_confidence(conf,0.96)):
        if(len(p[0:i+1])< len(p)):
            get_association_rule(p,t,p[0:i+1],i+1)
        else: 
            print(f'NESSUNA REGOLA per {p}')
            #with open("./rules"+"p{:02d}".format(i)+".txt", "a") as file:
            #    file.write(f'NESSUNA REGOLA per {p}\n')

    else:
        print(f'Rule:{ant} -> {p[i-1:-1]}: confidence {conf}')
        with open("./rules.txt", "a") as file:
                file.write(f'Rule:{ant} -> {p[i-1:-1]}: confidence {conf}\n')
        

In [30]:
def get_rules(patterns,tr):
    """Funzione che richiama la funzione che genera le regole per ogni partecipante.
    
    Parametri:
    - patterns: lista di pattern
    - tr: lista di transazioni"""
    for p in patterns:
        #richiamare una funzione che mi sposta la freccia e calcola la confidenza della regola
        ant = []
        ant.append(p[0])
        get_association_rule(p,tr,ant,1)
    

In [31]:
from os.path import exists
import os

In [32]:
# Eseguire per stampare e generare le regole
if(exists("./rules.txt")):
    os.remove("rules.txt")
for i in range(len(patterns)):
    print(f"\n\nREGOLE PER PARTECIPANTE {i+1}\n")
    with open("./rules.txt", "a") as file:
        file.write(f"\n\nREGOLE PER PARTECIPANTE {i+1}\n")
    get_rules(patterns[i],transactions[i])



REGOLE PER PARTECIPANTE 1

Rule:['calories cluster: 3, weekend: False'] -> ['calories cluster: 3, weekend: False']: confidence 0.9722222222222222
Rule:['calories cluster: 3, weekend: False'] -> ['calories cluster: 3, weekend: False']: confidence 0.9722222222222222
Rule:['calories cluster: 3, weekend: False', 'calories cluster: 0, weekend: False'] -> ['calories cluster: 0, weekend: False']: confidence 0.980952380952381
Rule:['calories cluster: 3, weekend: False', 'calories cluster: 0, weekend: False', 'calories cluster: 0, weekend: False'] -> ['calories cluster: 0, weekend: False']: confidence 0.9611650485436892
Rule:['calories cluster: 1, weekend: False'] -> ['calories cluster: 1, weekend: False']: confidence 1.0
Rule:['calories cluster: 1, weekend: False'] -> ['calories cluster: 1, weekend: False', 'calories cluster: 3, weekend: False']: confidence 0.9722222222222222
Rule:['calories cluster: 1, weekend: False'] -> ['calories cluster: 1, weekend: False', 'calories cluster: 3, weekend